In [ ]:
from autograd import grad
import torch
import autograd.numpy as np
import scipy.stats as st
import numpy as np


def hamiltonian_monte_carlo(n_samples, negative_log_prob, initial_position, C, Q, gamma, path_len=1, step_size=0.5):
    """Run Hamiltonian Monte Carlo sampling.

    Parameters
    ----------
    n_samples : int
        Number of samples to return
    negative_log_prob : callable
        The negative log probability to sample from
    initial_position : np.array
        A place to start sampling from.
    path_len : float
        How long each integration path is. Smaller is faster and more correlated.
    step_size : float
        How long each integration step is. Smaller is slower and more accurate.

    Returns
    -------
    np.array
        Array of length `n_samples`.
    """
    # autograd magic
    #dVdq = grad(negative_log_prob)
    initial_position = torch.flatten(initial_position)

    # collect all our samples in a list
    samples = [initial_position]

    # Keep a single object for momentum resampling
    momentum = st.norm(0, 1)

    # If initial_position is a 10d vector and n_samples is 100, we want
    # 100 x 10 momentum draws. We can do this in one call to momentum.rvs, and
    # iterate over rows
    size = (n_samples,torch.numel(initial_position)) 
    for p0 in momentum.rvs(size=size):
        # Integrate over our path to get a new position and momentum
        q_new, p_new = leapfrog(
            samples[-1],
            p0,
            negative_log_prob,
            C,
            Q,
            gamma,
            path_len=path_len,
            step_size=step_size,
        )

        # Check Metropolis acceptance criterion
        start_log_p = negative_log_prob(torch.reshape(samples[-1],(4,4)), C, Q, gamma) - np.sum(momentum.logpdf(p0))
        new_log_p = negative_log_prob(torch.reshape(q_new,(4,4)), C, Q, gamma) - np.sum(momentum.logpdf(p_new))
        if np.log(np.random.rand()) < start_log_p - new_log_p:
            samples.append(q_new)
        else:
            samples.append((samples[-1]).clone())

    return samples[1:]


In [ ]:
def leapfrog(q, p, negative_log_prob, C, Q, gamma, path_len, step_size):
    """Leapfrog integrator for Hamiltonian Monte Carlo.

    Parameters
    ----------
    q : np.floatX
        Initial position
    p : np.floatX
        Initial momentum
    dVdq : callable
        Gradient of the velocity
    path_len : float
        How long to integrate for
    step_size : float
        How long each integration step should be

    Returns
    -------
    q, p : np.floatX, np.floatX
        New position and momentum
    """
    q, p = torch.tensor(q, requires_grad=True), torch.tensor(p)
    NLL =  negative_log_prob(torch.reshape(q, (4,4)), C, Q, gamma) #todo: fix
    NLL.backward()
    dVdq = torch.flatten(q.grad)
    
    print("initial q", q)
    
    p -= step_size * dVdq / 2  # half step
    
    q.grad.data.zero_()
    
    for _ in range(int(path_len / step_size) - 1):
        
        NLL =  negative_log_prob(torch.reshape(q, (4,4)), C, Q, gamma)
        NLL.backward()
        dVdq = torch.flatten(q.grad)
        
        with torch.no_grad():
            q += step_size * p  # whole step
        p -= step_size * dVdq  # whole step
        
        q.grad.data.zero_()
        
    NLL =  negative_log_prob(torch.reshape(q, (4,4)), C, Q, gamma)
    NLL.backward()
    dVdq = torch.flatten(q.grad)    
    
    with torch.no_grad():
        q += step_size * p  # whole step
    p -= step_size * dVdq / 2  # half step
    
    q.grad.data.zero_()
    
    print("final q", q)

    # momentum flip at end
    return q, -p

In [ ]:
# Take 1: let's do this without inforcing reversibility         
def compute_D(Q, x):
    D = torch.zeros(len(Q)-1)
    deltaQ2s = []
    for i in range(len(Q)-1):
        deltaQ2 = (Q[i+1]-Q[i])**2
        D[i] = deltaQ2 * (x[i][i+1]*x[i+1][i])**(0.5)
        deltaQ2s.append(deltaQ2)
        
    return D, np.mean(deltaQ2s)

def compute_smoothness_correction(D, gamma, Qvar):
    correction = 0
    for i in range(len(D)-1):
        correction += (D[i]-D[i+1])**2 / (2*(Qvar*gamma)**2)
        
    return correction

def negative_log_prob(x, C, Q, gamma):
    D, Qvar = compute_D(Q,x)
    #print('diffusion coefficients', D)
    #print('Qvariance', Qvar)
    smoothness_correction = compute_smoothness_correction(D, gamma, Qvar)
    print('smoothness correction', smoothness_correction)
    
    return - (C * torch.log(x)).sum() - smoothness_correction

In [ ]:
Q = torch.tensor([0., 0.1, 0.2, 0.3])
C = torch.tensor([[1000., 50., 20., 10.],[48., 50., 1., 1.],[1., 1., 600., 40.],[9., 7., 33., 300.]])
X0 = ((C + C.T)/(2*C.sum())).clone().detach().requires_grad_(True)
print(X0)

In [ ]:
print(torch.numel(X0))
samples = hamiltonian_monte_carlo(10, negative_log_prob, X0, C, Q, 0.01)

In [ ]:
NLL = negative_log_prob(X0, C, Q, 0.01)
print(NLL)
NLL.backward()
gradient = X0.grad
print(gradient)
X0.grad.data.zero_()

In [ ]:
print(samples)

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import theano.tensor as tt

In [ ]:
n = 4

mu1 = np.ones(n) * (1.0 / 2)
mu2 = -mu1

stdev = 0.1
sigma = np.power(stdev, 2) * np.eye(n)
isigma = np.linalg.inv(sigma)
dsigma = np.linalg.det(sigma)

w1 = 0.1  # one mode with 0.1 of the mass
w2 = 1 - w1  # the other mode with 0.9 of the mass


def two_gaussians(x):
    log_like1 = (
        -0.5 * n * tt.log(2 * np.pi)
        - 0.5 * tt.log(dsigma)
        - 0.5 * (x - mu1).T.dot(isigma).dot(x - mu1)
    )
    log_like2 = (
        -0.5 * n * tt.log(2 * np.pi)
        - 0.5 * tt.log(dsigma)
        - 0.5 * (x - mu2).T.dot(isigma).dot(x - mu2)
    )
    return pm.math.logsumexp([tt.log(w1) + log_like1, tt.log(w2) + log_like2])

In [ ]:
with pm.Model() as model:
    X = pm.Uniform(
        "X",
        shape=n,
        lower=-2.0 * np.ones_like(mu1),
        upper=2.0 * np.ones_like(mu1),
        testval=-1.0 * np.ones_like(mu1),
    )
    llk = pm.Potential("llk", two_gaussians(X))
    trace = pm.sample_smc(2000, parallel=True)
    az_trace = az.from_pymc3(trace)

In [ ]:
ax = az.plot_trace(az_trace, compact=True, kind="rank_vlines")
ax[0, 0].axvline(-0.5, 0, 0.9, color="k")

ax[0, 0].axvline(0.5, 0, 0.1, color="k");

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint

C = np.array([[1000., 50., 20., 10.],[48., 50., 1., 1.],[1., 1., 600., 40.],[9., 7., 33., 300.]])

def random_initialiser(C):
    dim = int(C.shape[0] * (C.shape[0] + 1) / 2)
    x0 = np.random.uniform(size=dim)
    
    return x0

def extract_symmetric_components(matrix):
    dim = matrix.shape[0]
    symmetric_components = []
    for i in range(dim):
        for j in range(dim):
            if j >= i:
                symmetric_components.append(matrix[i][j])
                
    return np.array(symmetric_components)


def compute_X(symmetric_components):
    dim = int((-1+np.sqrt(1+8*symmetric_components.size))/2)
    X = np.zeros((dim, dim))
    for i in range(dim):
        for j in range(dim):
            if j >= i:
                X[i][j] = symmetric_components[int(i*dim - i*(i-1)/2 + j - i)]
                X[j][i] = X[i][j]  # symmetrise
                
    return X

def compute_D(Q, X, x):
    D = np.zeros(len(Q) - 1)
    deltaQ2s = []
    for i in range(len(Q) - 1):
        deltaQ2 = (Q[i + 1] - Q[i]) ** 2
        D[i] = deltaQ2 * (X[i][i + 1]*X[i+1][i] / x[i]*x[i+1]) ** (0.5)
        deltaQ2s.append(deltaQ2)

    return D, np.mean(deltaQ2s)


def compute_smoothness_correction(D, gamma, Qvar):
    correction = 0
    for i in range(len(D) - 1):
        correction += (D[i] - D[i + 1]) ** 2 / (2 * (Qvar * gamma) ** 2)

    return correction

def negative_log_likelihood(X, C, Q, gamma, t, use_smoothing):
    X = compute_X(symmetric_components = X)
    x = np.sum(X, axis=1) #col sum

    if use_smoothing:
        D, Qvar = compute_D(Q, X, x)
        smoothness_correction = compute_smoothness_correction(D, gamma, Qvar)
    else:
        smoothness_correction = 0
        
    A = X/x
    
    positive_row_barrier_function = (1/t)*(np.sum(- np.log(x))+np.sum(- np.log(1-x)))
    
    nll = float(- np.sum(C * np.log(A)) + smoothness_correction + positive_row_barrier_function)

    return nll

In [ ]:
import matplotlib.pyplot as plt

Q = np.array([0., 0.1, 0.2, 0.3])
C = np.array([[1000., 50., 20., 10.],[48., 50., 1., 1.],[1., 1., 600., 40.],[9., 7., 33., 300.]])
gamma = 0.001
use_smoothing=True
t = 1
X0 = np.random.uniform(0.8, 1.2, size=(C.shape))*((C + C.T)/(2*np.sum(C)))


number_of_parameters = int(C.shape[0]*(C.shape[0]+1)/2)
A = np.eye(number_of_parameters)
b = np.zeros(number_of_parameters)
lincon = LinearConstraint(A, b, np.inf*np.ones(number_of_parameters), keep_feasible=True)
    
x0 = extract_symmetric_components(X0)
print(np.sum(X0, axis=1))
print(compute_X(x0)/np.sum(compute_X(x0), axis=1))

for gamma in [0.1,0.01,0.001]:
    res = minimize(method='Nelder-Mead', fun=negative_log_likelihood, x0 = x0, args=(C, Q, gamma, t, use_smoothing), constraints=(lincon,), options={'maxiter': 20000})
    #print(res)
    X_star = compute_X(res.x)
    x_star = np.sum(X_star, axis=1)
    pi_star = x_star/np.sum(x_star)
    #print(pi_star)
    P_star = X_star/x_star
    plt.imshow(P_star)
    plt.colorbar()
    #print("gamma", gamma)
    #print(P_star)
    #print(x_star)
    #print(np.sum(x_star))
    #print(X_star)
    #print(x_star)
    print(P_star)
    for i in range(X.shape[0]-1):
        print(P_star[i][i + 1]*P_star[i+1][i])
    D, Qvar = compute_D(Q, X_star, x_star)
    plt.show()
    plt.plot(D)
    plt.show()